In [1]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset

In [2]:
from senmodel.model.utils import convert_dense_to_sparse_network
from senmodel.metrics.edge_finder import EdgeFinder


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from sklearn.preprocessing import LabelEncoder

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)
data = data.dropna()

scaler = StandardScaler()

y = data['occupation']
y = LabelEncoder().fit_transform(y)

X = data.drop(['occupation'], axis=1)

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

len(set(y))

15

In [5]:
class TabularDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [6]:
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

In [7]:
class MulticlassFCN(nn.Module):
    def __init__(self, input_size=14, hidden_sizes=None, output_size=15, dropout_rate=0.3):
        super(MulticlassFCN, self).__init__()
        if hidden_sizes is None:
            hidden_sizes = [128, 64]
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.1)

        self.output = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.output(x)
        # x = self.dropout2(x)
        return x


In [8]:
def edge_replacement_func_new_layer(model, optim, val_loader, metric, choose_threshold):
    layer = get_model_last_layer(model)
    ef = EdgeFinder(metric, val_loader, device)
    vals = ef.calculate_edge_metric_for_dataloader(model)
    print("Edge metrics:", vals, max(vals), sum(vals))
    chosen_edges = ef.choose_edges_threshold(model, choose_threshold)
    print("Chosen edges:", chosen_edges, len(chosen_edges[0]))
    layer.replace_many(*chosen_edges)

    if layer.embed_linears:
        optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
    else:
        print("Empty metric")
        dummy_param = torch.zeros_like(layer.weight_values)
        optim.add_param_group({'params': dummy_param})

    return {'max': max(vals), 'sum': sum(vals), 'len': len(vals), 'len_choose': len(chosen_edges[0])}


In [9]:
from senmodel.model.utils import freeze_all_but_last
from tqdm import tqdm
import torch.optim as optim
from sklearn.metrics import accuracy_score


def train_sparse_recursive(model, train_loader, val_loader, num_epochs, metric, edge_replacement_func=None,
                           window_size=3, threshold=0.1, lr=5e-4, choose_threshold=0.3, accumulation_steps=4):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        optimizer.zero_grad()

        for i, (inputs, targets) in enumerate(tqdm(train_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            loss = loss / accumulation_steps
            loss.backward()

            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            train_loss += loss.item() * accumulation_steps

        if len(train_loader) % accumulation_steps != 0:
            optimizer.step()
            optimizer.zero_grad()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        val_accuracy = accuracy_score(all_targets, all_preds)
        print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

        new_l = {}
        val_losses.append(val_loss)
        if edge_replacement_func and len(val_losses) > window_size:
            recent_changes = [abs(val_losses[i] - val_losses[i - 1]) for i in range(-window_size, 0)]
            avg_change = sum(recent_changes) / window_size
            if avg_change < threshold:
                new_l = edge_replacement_func(model, optimizer, val_loader, metric, choose_threshold)
                # Замораживаем все слои кроме последнего
                freeze_all_but_last(model,new_l['len_choose'])

        wandb.log({'val_loss': val_loss, 'val_accuracy': val_accuracy, 'train_loss': train_loss} | new_l)

In [10]:
from senmodel.metrics.nonlinearity_metrics import *

criterion = nn.CrossEntropyLoss()
metrics = [
    GradientMeanEdgeMetric(criterion),
    SNIPMetric(criterion),
    MagnitudeL2Metric(criterion),
]

In [11]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fedornigretuk. Use `wandb login --relogin` to force relogin


True

In [12]:
hyperparams = {"num_epochs": 50,
               "metric": metrics[0],
               "choose_threshold": 0.05,
               "window_size": 3,
               "threshold": 0.05,
               "lr": 5e-4,
               "accumulation_steps": 1,
               }

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)
name

'num_epochs: 50, metric: GradientMeanEdgeMetric, choose_threshold: 0.05, window_size: 3, threshold: 0.05, lr: 0.0005, accumulation_steps: 1'

In [13]:
dense_model = MulticlassFCN(input_size=X.shape[1])
sparse_model = convert_dense_to_sparse_network(dense_model)
wandb.finish()
wandb.init(
    project="self-expanding-nets",
    name=f"titanic-mul, {name}",
    tags=["complex model", "titanic", "multiclass", hyperparams["metric"].__class__.__name__],
    group="new freeze 2"
)

train_sparse_recursive(sparse_model, train_loader, val_loader,
                       edge_replacement_func=edge_replacement_func_new_layer, **hyperparams)
wandb.finish()

100%|██████████| 51/51 [00:01<00:00, 39.70it/s]


Epoch 1/50 | Train Loss: 2.5428 | Val Loss: 30.7755 | Val Accuracy: 0.2493


100%|██████████| 51/51 [00:01<00:00, 36.31it/s]


Epoch 2/50 | Train Loss: 2.3065 | Val Loss: 27.9201 | Val Accuracy: 0.3267


100%|██████████| 51/51 [00:01<00:00, 37.16it/s]


...

Epoch 46/50 | Train Loss: 1.9714 | Val Loss: 24.6049 | Val Accuracy: 0.3499
Edge metrics: tensor([0.0000e+00, 0.0000e+00, 4.7978e-03, 1.8179e-02, 3.5028e-02, 8.3300e-03,
        1.4317e-02, 2.1041e-04, 1.2703e-02, 1.0529e-02, 1.3901e-02, 2.1669e-02,
        7.1350e-03, 4.4520e-03, 4.2353e-02, 1.4805e-02, 2.1218e-02, 3.0938e-02,
        3.2529e-02, 4.2138e-02, 2.2772e-02, 2.0735e-02, 3.4303e-03, 4.3462e-02,
        2.0558e-02, 2.0230e-02, 1.8316e-02, 3.7753e-02, 1.2431e-02, 2.1336e-03,
        1.5668e-03, 2.6614e-02, 4.0156e-02, 1.3608e-03, 1.7253e-02, 0.0000e+00,
        1.5977e-02, 6.2006e-03, 4.1305e-02, 1.9743e-02, 2.1228e-02, 1.8667e-03,
        3.8980e-02, 2.0100e-02, 2.4880e-03, 3.4261e-02, 1.2673e-02, 2.6816e-02,
        6.3635e-03, 2.1550e-02, 1.8260e-02, 2.1657e-02, 1.5628e-03, 2.3539e-02,
        3.2134e-02, 6.5857e-03, 1.1145e-02, 3.7784e-03, 7.4729e-03, 2.7812e-03,
        3.6536e-02, 6.3060e-03, 0.0000e+00, 0.0000e+00, 3.9683e-02, 0.0000e+00,
        3.3065e-02, 0.0000e+00

100%|██████████| 51/51 [00:19<00:00,  2.66it/s]


Epoch 47/50 | Train Loss: 1.9690 | Val Loss: 24.6098 | Val Accuracy: 0.3531
Edge metrics: tensor([0.0000e+00, 0.0000e+00, 4.9417e-03, 1.9791e-02, 3.9095e-02, 8.6069e-03,
        1.6013e-02, 2.1552e-04, 1.3461e-02, 1.1052e-02, 1.5374e-02, 2.3858e-02,
        7.3607e-03, 4.5590e-03, 4.7396e-02, 1.5345e-02, 2.1891e-02, 3.4022e-02,
        3.6523e-02, 4.7600e-02, 2.5589e-02, 2.2025e-02, 3.5444e-03, 4.9142e-02,
        2.2557e-02, 2.3053e-02, 1.8919e-02, 4.2739e-02, 1.2849e-02, 2.1870e-03,
        1.6157e-03, 2.9780e-02, 4.5641e-02, 1.4091e-03, 1.8165e-02, 0.0000e+00,
        1.7660e-02, 6.3998e-03, 4.7076e-02, 2.2176e-02, 2.3556e-02, 1.9328e-03,
        4.4098e-02, 2.0767e-02, 2.5518e-03, 3.8454e-02, 1.3045e-02, 3.0101e-02,
        6.5742e-03, 2.3687e-02, 1.9291e-02, 2.3543e-02, 1.6157e-03, 2.5878e-02,
        3.6086e-02, 6.8177e-03, 1.1537e-02, 3.8803e-03, 7.7178e-03, 2.8667e-03,
        4.1240e-02, 6.5036e-03, 0.0000e+00, 0.0000e+00, 4.2287e-02, 0.0000e+00,
        3.3959e-02, 0.0000e+00

 78%|███████▊  | 40/51 [00:17<00:04,  2.35it/s]


KeyboardInterrupt: 